In [1]:
%load_ext autoreload
%autoreload 2
from get_data import *
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.preprocessing import StandardScaler
from pickle import load, dump
import math
from sklearn.metrics import mean_squared_error
'''
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')
tf.config.experimental.set_memory_growth(device=gpus[0], enable=True)
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
'''

import torch
import torch.nn as nn


In [2]:
'''
download_data(
    start_date="2021-01-01",
    end_date="2022-02-28",
    currency_symbol="BTCUSDT",
    path="data/train_15_2021-01-01_2022-02-28",
    ohlcv_size='15m'
)

download_data(
    start_date="2021-01-01",
    end_date="2022-02-28",
    currency_symbol="BTCUSDT",
    path="data/train_15_2021-01-01_2022-02-28",
    ohlcv_size='15m'
)

download_data(
    start_date="2022-03-01",
    end_date="2022-03-30",
    currency_symbol="BTCUSDT",
    path="data/test_15_2022-03-01_2022-03-30",
    ohlcv_size='15m'
)


'''



'\ndownload_data(\n    start_date="2021-01-01",\n    end_date="2022-02-28",\n    currency_symbol="BTCUSDT",\n    path="data/train_15_2021-01-01_2022-02-28",\n    ohlcv_size=\'15m\'\n)\n\ndownload_data(\n    start_date="2021-01-01",\n    end_date="2022-02-28",\n    currency_symbol="BTCUSDT",\n    path="data/train_15_2021-01-01_2022-02-28",\n    ohlcv_size=\'15m\'\n)\n\ndownload_data(\n    start_date="2022-03-01",\n    end_date="2022-03-30",\n    currency_symbol="BTCUSDT",\n    path="data/test_15_2022-03-01_2022-03-30",\n    ohlcv_size=\'15m\'\n)\n\n\n'

In [3]:
PATH_TRAIN = "data/train_15_2021-01-01_2022-02-28"
PATH_TEST = "data/test_15_2022-03-01_2022-03-30"
SCALER_PATH = "scalers/scaler_15m.pkl"

SEQ_LEN = 40
BATCH_SIZE = 8

In [4]:
class AlgoTradingDataset(Dataset):
    def __init__(self, seq_len, path, scaler_path):
        self.scaler = load(open(scaler_path, 'rb'))
        tmp_X = get_high_from_data(path).reshape(-1, 1)
        tmp_X = self.scaler.transform(tmp_X).reshape(-1)
        self.X = []
        self.y = []

        for i in range(SEQ_LEN, len(tmp_X)):
            self.X.append(tmp_X[i - SEQ_LEN : i])
            self.y.append(tmp_X[i])
        self.X = torch.Tensor(self.X).unsqueeze(dim=-1)
        self.y = torch.Tensor(self.y)


    def __len__(self):
        return self.X.shape[0]

    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    

In [5]:
training_data = AlgoTradingDataset(SEQ_LEN, PATH_TRAIN, SCALER_PATH)
test_data = AlgoTradingDataset(SEQ_LEN, PATH_TEST, SCALER_PATH)
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
print(len(train_dataloader))
print(next(iter(train_dataloader))[0].shape)
print(next(iter(train_dataloader))[1].shape)
print(next(iter(test_dataloader))[0].shape)
print(next(iter(test_dataloader))[1].shape)

5063
torch.Size([8, 40, 1])
torch.Size([8])
torch.Size([8, 40, 1])
torch.Size([8])


<ipython-input-4-40b483053662>:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  self.X = torch.Tensor(self.X).unsqueeze(dim=-1)


In [6]:
class SuperBot(torch.nn.Module):
    def __init__(self, input_size, hidden_size, bidirectional=True, dropout_rate=0.1):
        super(SuperBot, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, dropout=dropout_rate, batch_first=True, bidirectional=True)
        if bidirectional:
            self.l1 = nn.Linear(hidden_size * 2, 1)
        else:
            self.l1 = nn.Linear(hidden_size, 1)

    def forward(self, inputs):
        output = self.lstm1(inputs)[0]

        output = self.l1(output)

        return output

In [7]:
def train(
    model,
    optimizer,
    criterion,
    epochs,
    train_data,
    log_loss_every=150
):
    running_loss = 0.0
    for epoch in tqdm(range(epochs)):
        for i, data in enumerate(train_data):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % log_loss_every == log_loss_every - 1:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / float(log_loss_every):.6f}')
                running_loss = 0.0

    print('Finished Training')


In [8]:
model = SuperBot(
    input_size=1,
    hidden_size=10,
    dropout_rate=0
)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
criterion = nn.MSELoss()

train(
    model,
    optimizer,
    criterion,
    epochs=10,
    train_data=train_dataloader
)

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\dmele\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,   150] loss: 0.012114
[1,   300] loss: 0.001012
[1,   450] loss: 0.000840
[1,   600] loss: 0.001485
[1,   750] loss: 0.001617
[1,   900] loss: 0.001237
[1,  1050] loss: 0.000953
[1,  1200] loss: 0.000561
[1,  1350] loss: 0.001129
[1,  1500] loss: 0.000965
[1,  1650] loss: 0.001029
[1,  1800] loss: 0.001129
[1,  1950] loss: 0.000678
[1,  2100] loss: 0.000684
[1,  2250] loss: 0.000505
[1,  2400] loss: 0.000166
[1,  2550] loss: 0.000506
[1,  2700] loss: 0.000486
[1,  2850] loss: 0.000369
[1,  3000] loss: 0.000762
[1,  3150] loss: 0.000467
[1,  3300] loss: 0.000684
[1,  3450] loss: 0.000711
[1,  3600] loss: 0.000865
[1,  3750] loss: 0.000677
[1,  3900] loss: 0.000743
[1,  4050] loss: 0.001332
[1,  4200] loss: 0.000581
[1,  4350] loss: 0.000364
[1,  4500] loss: 0.000238
[1,  4650] loss: 0.000500
[1,  4800] loss: 0.000512
[1,  4950] loss: 0.000308


 10%|█         | 1/10 [00:59<08:54, 59.35s/it]

[2,   150] loss: 0.003190
[2,   300] loss: 0.000876
[2,   450] loss: 0.000675
[2,   600] loss: 0.001193
[2,   750] loss: 0.001414
[2,   900] loss: 0.000997
[2,  1050] loss: 0.000713
[2,  1200] loss: 0.000317
[2,  1350] loss: 0.000936
[2,  1500] loss: 0.000826
[2,  1650] loss: 0.000888
[2,  1800] loss: 0.001073
[2,  1950] loss: 0.000617
[2,  2100] loss: 0.000638
[2,  2250] loss: 0.000467
[2,  2400] loss: 0.000128
[2,  2550] loss: 0.000460
[2,  2700] loss: 0.000440
[2,  2850] loss: 0.000308
[2,  3000] loss: 0.000695
[2,  3150] loss: 0.000406
[2,  3300] loss: 0.000639
[2,  3450] loss: 0.000656
[2,  3600] loss: 0.000781
[2,  3750] loss: 0.000604
[2,  3900] loss: 0.000681
[2,  4050] loss: 0.001281
[2,  4200] loss: 0.000537
[2,  4350] loss: 0.000324
[2,  4500] loss: 0.000213
[2,  4650] loss: 0.000480
[2,  4800] loss: 0.000495
[2,  4950] loss: 0.000287


 20%|██        | 2/10 [01:58<07:55, 59.42s/it]

...

[9,   150] loss: 0.002703
[9,   300] loss: 0.000798
[9,   450] loss: 0.000575
[9,   600] loss: 0.001010
[9,   750] loss: 0.001290
[9,   900] loss: 0.000901
[9,  1050] loss: 0.000613
[9,  1200] loss: 0.000237
[9,  1350] loss: 0.000845
[9,  1500] loss: 0.000741
[9,  1650] loss: 0.000800
[9,  1800] loss: 0.000960
[9,  1950] loss: 0.000521
[9,  2100] loss: 0.000561
[9,  2250] loss: 0.000397
[9,  2400] loss: 0.000113
[9,  2550] loss: 0.000407
[9,  2700] loss: 0.000413
[9,  2850] loss: 0.000284
[9,  3000] loss: 0.000600
[9,  3150] loss: 0.000348
[9,  3300] loss: 0.000564
[9,  3450] loss: 0.000599
[9,  3600] loss: 0.000669
[9,  3750] loss: 0.000535
[9,  3900] loss: 0.000634
[9,  4050] loss: 0.001172
[9,  4200] loss: 0.000447
[9,  4350] loss: 0.000280
[9,  4500] loss: 0.000197
[9,  4650] loss: 0.000435
[9,  4800] loss: 0.000441
[9,  4950] loss: 0.000263


 90%|█████████ | 9/10 [08:55<00:59, 59.58s/it]

[10,   150] loss: 0.002665
[10,   300] loss: 0.000788
[10,   450] loss: 0.000567
[10,   600] loss: 0.000998
[10,   750] loss: 0.001278
[10,   900] loss: 0.000893
[10,  1050] loss: 0.000607
[10,  1200] loss: 0.000234
[10,  1350] loss: 0.000837
[10,  1500] loss: 0.000733
[10,  1650] loss: 0.000794
[10,  1800] loss: 0.000950
[10,  1950] loss: 0.000514
[10,  2100] loss: 0.000555
[10,  2250] loss: 0.000391
[10,  2400] loss: 0.000112
[10,  2550] loss: 0.000403
[10,  2700] loss: 0.000410
[10,  2850] loss: 0.000282
[10,  3000] loss: 0.000593
[10,  3150] loss: 0.000345
[10,  3300] loss: 0.000558
[10,  3450] loss: 0.000593
[10,  3600] loss: 0.000660
[10,  3750] loss: 0.000530
[10,  3900] loss: 0.000629
[10,  4050] loss: 0.001162
[10,  4200] loss: 0.000440
[10,  4350] loss: 0.000278
[10,  4500] loss: 0.000196
[10,  4650] loss: 0.000431
[10,  4800] loss: 0.000436
[10,  4950] loss: 0.000261


100%|██████████| 10/10 [09:54<00:00, 59.49s/it]

Finished Training


In [9]:
torch.save(model.state_dict(), "saved_models/lstm_v1.pth")